In [1]:
# data downloading
import data_util
import pandas as pd
from collections import defaultdict
import os
import sys

os.chdir("/home/yxjiang/source/ml_playground")
print(os.getcwd())

from config import *

/home/yxjiang/source/ml_playground


In [2]:
dataset_url="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dest_dir = "/tmp/data"
dataset_folder_path = os.path.join(dest_dir, "aclImdb")
data_util.download_data(url=dataset_url, dest_dir=dest_dir)

# generate word to id mapping
word_to_id, word_list = data_util.get_vocabulary(folder_path=dataset_folder_path, file_suffix="vocab")
print("There size of vocabulary is :", len(word_to_id))

Destination folder [/tmp/data] exists.
Target file [aclImdb_v1.tar.gz] exists, skip downloading.
Start to extract [/tmp/data/aclImdb_v1.tar.gz] to [/tmp/data]...
File extracted
Processing vocabulary from [/tmp/data/aclImdb].
There size of vocabulary is : 89527


In [3]:
# model
class RNN(nn.Module):
    def __init__(self, config, vocabulary_size):
        super().__init__()
        self.config = config
        self.rnn_type = config.rnn_type
        self.embed = nn.Embedding(vocabulary_size, config.word_embedding_length)
        self.num_layers = 1
        self.directions = 1
        self.hidden_size = 128
        if config.rnn_type is nn.RNN:
            self.rnn = nn.RNN(
                input_size=config.word_embedding_length, 
                hidden_size=self.hidden_size, 
                num_layers=self.num_layers
            )
        elif config.rnn_type is nn.LSTM:
            self.cell_size = 128
            self.rnn = nn.LSTM(
                input_size=config.word_embedding_length,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers
            )
        elif config.rnn_type is nn.GRU:
            self.rnn = nn.GRU(
                input_size=config.word_embedding_length, 
                hidden_size=self.hidden_size, 
                num_layers=self.num_layers
            )
        self.fc1 = nn.Linear(in_features=self.hidden_size, out_features=64)
        self.dropout = nn.Dropout(config.dropout)
        self.fc2 = nn.Linear(in_features=64, out_features=config.num_classes)

    def forward(self, x):
        batch = x.shape[0]
        x = self.embed(x)  # (batch, sentence_length, embedding_dim)
        x = x.permute(1, 0, 2).contiguous()  # (sentence_length, batch, embedding_dim)

        h0 = torch.zeros((self.num_layers * self.directions, batch, self.hidden_size)).to(self.config.device)
        if self.rnn_type is nn.RNN or self.rnn_type is nn.GRU:
            output, ht = self.rnn(x, h0)
        elif self.rnn_type is nn.LSTM:
            c0 = torch.zeros((self.num_layers * self.directions, batch, self.hidden_size)).to(self.config.device)
            output, (ht, ct) = self.rnn(x, (h0, c0))
        ht = ht.view(self.num_layers, self.directions, ht.shape[1], -1) # decompose layers and directions
        ht = ht[-1]  # get last layer and remove layer dimension
        ht = ht.permute(1, 0, 2)  # (batch, directions, embedding_dim)
        ht = ht.contiguous().view(batch, self.directions, self.hidden_size)
        x = F.relu(self.fc1(ht))
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.view(batch, -1)
        return x

In [6]:
# put everything together
import time
from data_util import *
from models import *
from trainer import classification_trainer

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# config = TextCNNConfig(
#             criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00003, epochs=1000, 
#             batch_size=1024, num_classes=2, sentence_max_length=30, word_embedding_length=128, 
#             activation=F.relu, dropout=0.1, conv_layer_sizes=[3,4,5,6,7]
#         )

def run(rnn_type, num_layers):
    config = RNNConfig(
        rnn_type=rnn_type,
        criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00001, epochs=500, 
        batch_size=1024, num_classes=2, sentence_max_length=40, word_embedding_length=128, 
        num_layers=num_layers, activation=F.relu, dropout=0.1,
    )
    print(config)

    pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
    neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
    train_dataset = MovieReviewDataset(config, pos_train_data_folder, neg_train_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                                ]))

    pos_test_data_folder = os.path.join(dataset_folder_path, "test/pos")
    neg_test_data_folder = os.path.join(dataset_folder_path, "test/neg")
    test_dataset = MovieReviewDataset(config, pos_test_data_folder, neg_test_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                                ]))

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)

    # model = TextCNN(config, len(word_to_id)).to(device)
    model = RNN(config, len(word_list)).to(config.device)

    classification_trainer.train(model, config, train_dataloader, test_dataloader)


run(nn.GRU, num_layers=1)
run(nn.RNN, num_layers=1)
run(nn.LSTM, num_layers=1)

sentence_max_len_40-embedding-128-model-<class 'torch.nn.modules.rnn.GRU'>-layers-5-lr-0.00001000-batch_size-1024-dropout-0.10
[1 seconds](epoch: 0/500)[25000 samples] loss: 0.701.
eval loss: 0.698, accuracy: 50.092% [12523/25000]
[97 seconds](epoch: 50/500)[1275000 samples] loss: 0.686.
eval loss: 0.692, accuracy: 52.372% [13093/25000]
[193 seconds](epoch: 100/500)[2525000 samples] loss: 0.682.
eval loss: 0.688, accuracy: 54.016% [13504/25000]
[289 seconds](epoch: 150/500)[3775000 samples] loss: 0.634.
eval loss: 0.646, accuracy: 62.596% [15649/25000]
[385 seconds](epoch: 200/500)[5025000 samples] loss: 0.585.
eval loss: 0.611, accuracy: 66.500% [16625/25000]
[480 seconds](epoch: 250/500)[6275000 samples] loss: 0.490.
eval loss: 0.592, accuracy: 68.452% [17113/25000]
[576 seconds](epoch: 300/500)[7525000 samples] loss: 0.493.
eval loss: 0.586, accuracy: 69.300% [17325/25000]
[671 seconds](epoch: 350/500)[8775000 samples] loss: 0.436.
eval loss: 0.590, accuracy: 69.748% [17437/25000]
[